pip install pandas

In [16]:
import pandas as pd

# Load the CSV file
csv_file = "lithuanian_words_1page.csv"
df = pd.read_csv(csv_file)

# Display basic info about the dataset
print("📝 Original Data Overview:")
print(df.info())

📝 Original Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Word             50 non-null     object
 1   Part of Speech   49 non-null     object
 2   Dark Text        50 non-null     object
 3   Description      50 non-null     object
 4   Additional Info  47 non-null     object
 5   Update Date      37 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB
None


In [26]:
from IPython.display import display
display(df.head())

,Word,Part of Speech,Dark Text,Description,Additional Info,Update Date
0,a1,♦,a,"rasti, galva, kas; meta, gale | lapė, matė |...","raidė, vartojama žymėti užpakalinės eilės žemu...",Atnaujinta: 2024-07-19
1,a2,jst.,ã,"Ã, kai̇̃p šálta! Ã, prisi̇̀menu, prisi̇̀me...",(ir tęsiamai) vartojamas reiškiant nusistebėji...,Atnaujinta: 2021-10-27
2,a3,jst.,à,"À, kai̇̃p tù mán nusibódai. À, tiẽk tõ...","vartojamas reiškiant nekantrumą, nepasitenkini...",NaN
3,a-,priešd.,a-,"aritmija, asimetrija, ateizmas, apolitiškas, ...","tarptautinė žodžio dalis, reiškianti ko neigim...",Atnaujinta: 2023-05-12
4,ą,NaN,ą,"kąsti, šąla, ką; ranką, žąsis | „Ą, ą“ yrà ...","raidė, vartojama žymėti užpakalinės eilės žemu...",NaN
